In [1]:
import math
from sympy import isprime
import numpy as np
from itertools import product
import math
from decimal import Decimal

#Factoriza un número usando la criba cuadrática el método de Gauss para resolver el sistema
def factorizacion_criba(n):
    #factorizacion es la factorizacion de números a^2 mod N que son B suaves
    #La factorizacion está expresada como la potencia a la que se eleva cada uno
    #primos de la lista lista_primos. numeros son los números xi con los que se
    #ha conseguido que F(xi) sea de la forma a^2 mod N es B-suave
    factorizacion, lista_primos, numeros = criba(n)
    if len(factorizacion) <= 0:
      return 1, n
    #Planteamos el sistema de ecuaciones convirtiendola a modulo 2 y trasponiendo
    matriz = np.transpose(np.asarray(factorizacion) % 2)
    #Conseguimos la matriz triangular superior, así como el número de columna de
    #las variables libres
    diagonal, var_libres = diagonal_matrix(matriz)
    if len(diagonal) <= 0:
      return 1, n
    #Inicializo un vector relleno de -1 de longitud el número de columnas de la matriz
    solucion_sistema = [-1]*len(diagonal[0])
    #Voy generando las posibles combinaciones (modulo 2, es decir 0's o 1's)
    #para luego asignarlas a las variables libres
    generador = generar_combinaciones(len(var_libres))
    while True:
      try:
        #Genero combinacion de de 0's y 1's
        combinacion = next(generador)
        #Voy asignando a cada variable libre el valor que le corresponde
        for i in range(len(var_libres)):
          solucion_sistema[var_libres[i]] = combinacion[i]
        #Consigo las dimensiones de mi matriz
        rows, cols = diagonal.shape
        #Voy calculando el valor de las variables pivote en funcion de las
        #variables de su derecha empezando por la ultima fila
        for i in range(rows - 1, -1, -1):
          for j in range(cols):
            if diagonal[i, j] != 0:
              #El valor de una variable pivote será la suma (modulo 2) de los
              #valores de las variables de su derecha
              suma = 0
              for k in range(j+1, cols):
                suma = suma + (diagonal[i, k]*solucion_sistema[k]) % 2
              solucion_sistema[j] = suma
              #Solo hay una variable pivote por fila
              break

        #En este momento se tiene la solucion del sistema en la variable solucion_sistema


        #producto es la mutiplicacion de los números xi con los que se
        #ha conseguido que F(xi) sea de la forma a^2 mod N es B-suave. Es lo que
        #se usará en el mcd. Se usa Decimal para aumentar el tamaño de los enteros
        #con los que se puede trabajar
        producto = Decimal(1)
        for i in range(len(numeros)):
            producto *= Decimal(int(numeros[i]) ** int(solucion_sistema[i]))
        producto = int(producto)


        #factorizacion_producto es el conjunto de numeros x que hacen que
        #(xi*xj*xk...)^2 mod N. Está expresado como el exponente de los primos
        #de lista_primos (el elevado al cuadrado está incluido)
        factorizacion_producto = [0] * len(factorizacion[0])
        for i, condicion in enumerate(solucion_sistema):
          if condicion == 1:
            factorizacion_producto = [x + y for x, y in zip(factorizacion_producto, factorizacion[i])]


        #K es el producto de los números de factorizacion_producto quitandole
        #el cuadrado. Es lo que usará en el mcd
        K = 1
        for num1, num2 in zip(lista_primos, factorizacion_producto):
          K *= num1 ** (num2/2)
        K = int(K)

        #calculo el mcd
        mcd = math.gcd(n, producto - K)
        #si el mcd es distinto de 1 o N he encontrado la solucion
        if mcd != 1 and mcd != n:
          return mcd, int(n/mcd)
      except StopIteration:
          break
    return 1, n

#Devuelve la factorizacion de los números que se reducen a 1 al hacer la criba
#cuadrática sobre n en forma de exponentes a los que se elevan los primos
#y la lista de primos que usa
#
def criba(n):
    #a=⌊√N ⌋ + 1,
    base = math.floor(math.sqrt(n)) +1
    #L(N)
    l_n = math.exp( math.sqrt( math.log(n) * math.log(math.log(n))) )
    #B=L(N)^1/√2
    B = math.floor(l_n**(1/math.sqrt(2))) + 1
    #b=L(N)^√2
    tope = math.floor( pow(l_n, math.sqrt(2)) ) + 1


    #Creo la lista de valores iniciales
    lista=[]
    for num in range(base, tope + 1):
      lista.append(num**2 - n)
    #Creo una lista de lista para guardar las divisiones que aplico
    divisores=[[] for _ in range(tope+1-base)]

    lista_primos=calcular_primos(B)
    #Hago un bucle para dividir por todos los primos y sus potencias
    for primo in lista_primos:
      #i será el exponente al que se eleva el primo
      i=1
      #Hago un bucle para dividir por todas las potencias
      while pow(primo, i) < B:
        #Consigo los valores que me resuelvan la ecuacion
        soluciones=resolver_ecuacion(base, tope, n, pow(primo, i))
        #Si no hay soluciones  paso al siguiente número
        if soluciones != []:
          #Hago un bucle para las dos soluciones
          for sol in soluciones:
            #Inicialmente j será la posicion en la lista en la que está la solución
            j = sol - base
            #Divido todos los elementos con posicion j+k*primo
            #y guardo la division
            while j < len(lista):
              potencia = i-cuenta_num_apariciones(divisores[j], primo)
              divisor = pow(primo, potencia)
              if divisor != 1:
                lista[j] /= divisor
                for _ in range(potencia):
                  divisores[j].append(divisor)
              j += primo
        i +=1
    #En posiciones se guardará las posiciones de los elementos de la lista que se
    #hayan reducido a 1
    posiciones = []
    for indice, elemento in enumerate(lista):
        if elemento == 1:
            posiciones.append(indice)

    #resultado_divisores es una lista de listas con los factores de los elementos
    #que se hayan reducido a 1
    resultado_divisores = [divisores[posicion] for posicion in posiciones]
    #Devuelvo la factorización de los elementos que se hayan reducido a 1 en forma
    #de exponentes de la lista de primos
    return [contar_apariciones(lista_primos, lista) for lista in resultado_divisores], lista_primos, [pos + base for pos in posiciones]

#Devuelve la matriz diagonal y
#la lista con el número de columna de las variables libres
def diagonal_matrix(M):
    A = np.copy(M)
    rows, cols = A.shape
    var_libres = list(range(cols))
    # Triangular superior
    for i in range(rows):
      #Busco el pivote de la fila i
      for j in range(cols):
        #Compruebo si es pivote
        if A[i, j] != 0:
          var_libres[j] = -1
          #Resto una unidad a las filas de abajo del pivote que tengan un 1 debajo del pivote
          for k in range(i+1, rows):
              if A[k, j] != 0:
                  A[k] = [ (x - y) % 2 for x, y in zip(A[k], A[i])]
          break
    return quitar_fila_ceros(A), [x for x in var_libres if x != -1]

#Calcula los número primos menores que B
def calcular_primos(B):
    primos = []
    i = 2
    while i <= B:
        if isprime(i):
            primos.append(i)
        i += 1
    return primos

#Devuelve las dos primeras soluciones entre el tope y la base que cumplan que t^2 = n mod p
def resolver_ecuacion(base, tope, n, p):
    soluciones = []
    N = n % p
    for t in range(base, tope + 1):
        if (t**2) % p == N:
            soluciones.append(t)
            if len(soluciones) == 2:
                return soluciones
    return soluciones

#Devuelve el número de apariciones de p en lista
def cuenta_num_apariciones(lista, p):
    count = 0
    for elemento in lista:
        if elemento == p:
            count += 1
    return count

#Devuelve el número de apariciones de los elementos de la primera lista en la segunda
def contar_apariciones(lista1, lista2):
    contador = []
    for elemento in lista1:
        contador.append(lista2.count(elemento))
    return contador

#Devuelve la matriz quitando las filas de todo ceros
def quitar_fila_ceros(matriz):
    return matriz[np.any(matriz, axis=1)]

#Genera las posibles combinaciones de 0's y 1's de longitud n segun se vayan necesitando
def generar_combinaciones(n):
    for combinacion in product([0, 1], repeat=n):
        yield combinacion





In [2]:
print(factorizacion_criba(165))

(11, 15)


In [3]:
print(factorizacion_criba(195))

(13, 15)


In [4]:
print(factorizacion_criba(1827))

(21, 87)


In [5]:
print(factorizacion_criba(1881))

(33, 57)


In [6]:
print(factorizacion_criba(2077))

(31, 67)


In [7]:
print(factorizacion_criba(6509))

(23, 283)


In [8]:
print(factorizacion_criba(11049))

(87, 127)


In [9]:
print(factorizacion_criba(9993))

(3, 3331)


In [10]:
print(factorizacion_criba(9788111))

(2741, 3571)
